# LSTM - By Industry

In [1]:
# Import dependencies
import numpy as np
import pandas as pd
import datetime

In [2]:
# Import learning model dependencies
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
pd.options.mode.chained_assignment = None
tf.random.set_seed(0)

In [3]:
# Establish connection to AWS server
from config import db_password
import psycopg2
connection = psycopg2.connect(
    host = 'launch-it-1.cyo6pvehqvyz.us-east-1.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = db_password,
    database='launch-it-1'
    )
cursor=connection.cursor()

In [4]:
# Pull SQL dataset from database
sql = """
SELECT * FROM launchit  
"""

# Build dataframe from SQL data
stock_data = pd.read_sql(sql, con=connection)
stock_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,TCKR
0,2019-12-31,87.300003,87.300003,64.904999,71.699997,71.699997,33947.0,MKDTY
1,2020-01-02,72.900002,72.900002,62.250000,65.550003,65.550003,13607.0,MKDTY
2,2020-01-03,63.299999,66.750000,63.299999,65.250000,65.250000,4427.0,MKDTY
3,2020-01-06,65.550003,72.000000,63.750000,67.650002,67.650002,9933.0,MKDTY
4,2020-01-07,68.250000,72.419998,68.099998,72.000000,72.000000,8347.0,MKDTY


In [5]:
# Pull SQL dataset from database
sql = """
SELECT * FROM launchit_info 
"""

# Build dataframe from SQL data
info_data = pd.read_sql(sql, con=connection)
info_data.head()

,TCKR,sector,industry,country,growth_rate,debtToEquity,currentRatio,forwardPE
0,MKDTY,Basic Materials,Chemicals,China,-0.671,NaN,0.928,-0.002444
1,INDO,Energy,Oil & Gas E&P,Indonesia,1.206,12.272,2.707,-9.695652
2,MNPR,Healthcare,Biotechnology,United States,NaN,0.167,7.508,-2.596638
3,OCFT,Technology,Software—Application,China,0.004,9.305,1.241,-3.550454
4,SPT,Technology,Software—Application,United States,0.330,16.084,1.886,-3289.500000


In [6]:
# Merge the stock_data and info_data
data_all = stock_data.merge(info_data, how='left', left_on='TCKR', right_on='TCKR')
data_all

,Date,Open,High,Low,Close,Adj Close,Volume,TCKR,sector,industry,country,growth_rate,debtToEquity,currentRatio,forwardPE
0,2019-12-31,87.300003,87.300003,64.904999,71.699997,71.699997,33947.0,MKDTY,Basic Materials,Chemicals,China,-0.671,NaN,0.928,-0.002444
1,2020-01-02,72.900002,72.900002,62.250000,65.550003,65.550003,13607.0,MKDTY,Basic Materials,Chemicals,China,-0.671,NaN,0.928,-0.002444
2,2020-01-03,63.299999,66.750000,63.299999,65.250000,65.250000,4427.0,MKDTY,Basic Materials,Chemicals,China,-0.671,NaN,0.928,-0.002444
3,2020-01-06,65.550003,72.000000,63.750000,67.650002,67.650002,9933.0,MKDTY,Basic Materials,Chemicals,China,-0.671,NaN,0.928,-0.002444
4,2020-01-07,68.250000,72.419998,68.099998,72.000000,72.000000,8347.0,MKDTY,Basic Materials,Chemicals,China,-0.671,NaN,0.928,-0.002444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671669,2022-11-08,10.270000,10.270000,10.270000,10.270000,10.270000,1500.0,CITE,Financial Services,Shell Companies,United States,NaN,NaN,9.277,NaN
671670,2022-11-09,10.260000,10.260000,10.230000,10.230000,10.230000,3500.0,CITE,Financial Services,Shell Companies,United States,NaN,NaN,9.277,NaN
671671,2022-11-10,10.230000,10.280000,10.230000,10.280000,10.280000,27400.0,CITE,Financial Services,Shell Companies,United States,NaN,NaN,9.277,NaN
671672,2022-11-11,10.280000,10.280000,10.280000,10.280000,10.280000,0.0,CITE,Financial Services,Shell Companies,United States,NaN,NaN,9.277,NaN


In [7]:
# Put the industry names in a list.
industries = pd.unique(data_all['industry']).tolist()

In [8]:
data_all = data_all.set_index(['Date' , 'industry'])
data_all

,,Open,High,Low,Close,Adj Close,Volume,TCKR,sector,country,growth_rate,debtToEquity,currentRatio,forwardPE
Date,industry,,,,,,,,,,,,,
2019-12-31,Chemicals,87.300003,87.300003,64.904999,71.699997,71.699997,33947.0,MKDTY,Basic Materials,China,-0.671,NaN,0.928,-0.002444
2020-01-02,Chemicals,72.900002,72.900002,62.250000,65.550003,65.550003,13607.0,MKDTY,Basic Materials,China,-0.671,NaN,0.928,-0.002444
2020-01-03,Chemicals,63.299999,66.750000,63.299999,65.250000,65.250000,4427.0,MKDTY,Basic Materials,China,-0.671,NaN,0.928,-0.002444
2020-01-06,Chemicals,65.550003,72.000000,63.750000,67.650002,67.650002,9933.0,MKDTY,Basic Materials,China,-0.671,NaN,0.928,-0.002444
2020-01-07,Chemicals,68.250000,72.419998,68.099998,72.000000,72.000000,8347.0,MKDTY,Basic Materials,China,-0.671,NaN,0.928,-0.002444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-08,Shell Companies,10.270000,10.270000,10.270000,10.270000,10.270000,1500.0,CITE,Financial Services,United States,NaN,NaN,9.277,NaN
2022-11-09,Shell Companies,10.260000,10.260000,10.230000,10.230000,10.230000,3500.0,CITE,Financial Services,United States,NaN,NaN,9.277,NaN
2022-11-10,Shell Companies,10.230000,10.280000,10.230000,10.280000,10.280000,27400.0,CITE,Financial Services,United States,NaN,NaN,9.277,NaN


In [9]:
# List all of the column names
data_all.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'TCKR', 'sector',
       'country', 'growth_rate', 'debtToEquity', 'currentRatio', 'forwardPE'],
      dtype='object')

In [10]:
# Create a new data frame grouped by Industry and taking the median of numeric values

# Make a new copy of the data_all 
industry_data = data_all.copy()

# Drop unneeded columns
industry_data = industry_data.drop(labels=['TCKR', 'sector', 'country', 'growth_rate', 'debtToEquity',
       'currentRatio', 'forwardPE'], axis=1)

# sort the data using sort_index()
industry_data.sort_index()
industry_data

,,Open,High,Low,Close,Adj Close,Volume
Date,industry,,,,,,
2019-12-31,Chemicals,87.300003,87.300003,64.904999,71.699997,71.699997,33947.0
2020-01-02,Chemicals,72.900002,72.900002,62.250000,65.550003,65.550003,13607.0
2020-01-03,Chemicals,63.299999,66.750000,63.299999,65.250000,65.250000,4427.0
2020-01-06,Chemicals,65.550003,72.000000,63.750000,67.650002,67.650002,9933.0
2020-01-07,Chemicals,68.250000,72.419998,68.099998,72.000000,72.000000,8347.0
...,...,...,...,...,...,...,...
2022-11-08,Shell Companies,10.270000,10.270000,10.270000,10.270000,10.270000,1500.0
2022-11-09,Shell Companies,10.260000,10.260000,10.230000,10.230000,10.230000,3500.0
2022-11-10,Shell Companies,10.230000,10.280000,10.230000,10.280000,10.280000,27400.0


In [11]:
# Group by Industry and take the median of all numeric values
# industry_data = industry_data.groupby(level='Date').median()

industry_data = industry_data.groupby(level=[0,1]).median()
industry_data

Open       High        Low  \
Date       industry                                                     
1992-05-21 Software—Infrastructure    20.000000  20.000000  17.750000   
1992-05-22 Software—Infrastructure    18.000000  18.500000  18.000000   
1992-05-26 Software—Infrastructure    18.000000  18.500000  16.500000   
1992-05-27 Software—Infrastructure    17.500000  17.500000  16.000000   
1992-05-28 Software—Infrastructure    16.500000  17.500000  16.500000   
...                                         ...        ...        ...   
2022-11-14 Utilities—Diversified      11.880000  12.710000  11.880000   
           Utilities—Regulated Gas    51.000000  51.145000  49.610001   
           Utilities—Regulated Water  17.450001  17.655001  16.950001   
           Utilities—Renewable         9.380000   9.829950   9.060000   
           Waste Management           37.840000  38.064999  36.660000   

                                          Close  Adj Close     Volume  
Date       industry                                                    
1992-05-21 Software—Infrastructure    18.000000  18.000000   248200.0  
1992-05-22 Software—Infrastructure    18.000000  18.000000    69150.0  
1992-05-26 Software—Infrastructure    17.000000  17.000000    84700.0  
1992-05-27 Software—Infrastructure    17.000000  17.000000    17300.0  
1992-05-28 Software—Infrastructure    17.000000  17.000000    40200.0  
...                                         ...        ...        ...  
2022-11-14 Utilities—Diversified      12.410000  12.410000   435482.0  
           Utilities—Regulated Gas    49.810001  49.810001  1154815.0  
           Utilities—Regulated Water  17.540001  17.540001   392709.0  
           Utilities—Renewable         9.725000   9.725000   263651.0  
           Waste Management           37.219999  37.219999   781582.5  

[89368 rows x 6 columns]

In [12]:
industry_data.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [13]:
# # Obtain CSV for Monika
# industry_data

# # Reset the index
# industry_data.reset_index(inplace=True)

# # Save as CSV file
# industry_data.to_csv('industry_data.csv', header=True, sep=',')

In [14]:
# Select one of the industries from this list
industries

['Chemicals',
 'Oil & Gas E&P',
 'Biotechnology',
 'Software—Application',
 'Aerospace & Defense',
 'Capital Markets',
 'Auto & Truck Dealerships',
 'REIT—Retail',
 'Computer Hardware',
 'Semiconductors',
 'Rental & Leasing Services',
 'Internet Content & Information',
 'Specialty Retail',
 'Medical Devices',
 'Diagnostics & Research',
 'Banks—Regional',
 'Real Estate Services',
 'Medical Care Facilities',
 'Education & Training Services',
 'Packaged Foods',
 'Health Information Services',
 'Insurance Brokers',
 'Credit Services',
 'Leisure',
 None,
 'Medical Instruments & Supplies',
 'Software—Infrastructure',
 'Information Technology Services',
 'Asset Management',
 'Restaurants',
 'Drug Manufacturers—Specialty & Generic',
 'Oil & Gas Drilling',
 'Electronic Gaming & Multimedia',
 'Solar',
 '',
 'Shell Companies',
 'Broadcasting',
 'Luxury Goods',
 'Communication Equipment',
 'Grocery Stores',
 'Internet Retail',
 'Specialty Chemicals',
 'REIT—Office',
 'Staffing & Employment Service

In [15]:
# Reset the index
industry_data.reset_index(inplace=True)

# Set the date column as the index
industry_data = industry_data.set_index('Date')

# Isolate just 1 industry to attempt LSTM process
df = industry_data[industry_data.industry == 'Oil & Gas E&P']
df

,industry,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2016-11-16,Oil & Gas E&P,39.750000,39.750000,38.009998,39.000000,38.913258,12600.0
2016-11-17,Oil & Gas E&P,39.000000,39.000000,35.000000,36.990002,36.907730,28300.0
2016-11-18,Oil & Gas E&P,38.950001,39.750000,34.750000,39.750000,39.661587,1100.0
2016-11-21,Oil & Gas E&P,40.000000,40.000000,35.060001,40.000000,39.911034,2500.0
2016-11-22,Oil & Gas E&P,40.000000,41.000000,36.000000,41.000000,40.908806,4100.0
...,...,...,...,...,...,...,...
2022-11-08,Oil & Gas E&P,23.989999,24.500000,23.210001,24.315001,24.315001,474150.0
2022-11-09,Oil & Gas E&P,23.974999,24.259999,23.170000,23.700001,23.700001,480900.0
2022-11-10,Oil & Gas E&P,24.275000,24.375000,23.795001,24.205000,24.205000,514900.0


In [16]:
# # Isolate just 1 Industry to attempt LSTM process
# df = industry_data.filter(like = 'Oil & Gas E&P', axis=0)

# # This method only works if the industry column is still incorporated into the index prior to resetting it.
# # Alternatively - just reset the index after the groupby function groups by industry and date.

# # Reset the index
# df.reset_index(inplace=True)

# # Set the date column as the index
# df = df.set_index('Date')
# df

In [17]:
## Run this to generate visualization of the industry of interest

# # download the data
# y = df['Adj Close'].fillna(method='ffill')
# y = y.values.reshape(-1, 1)

# # scale the data
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaler = scaler.fit(y)
# y = scaler.transform(y)

# # generate the input and output sequences
# n_lookback = 365  # length of input sequences (lookback period)
# n_forecast = 180  # length of output sequences (forecast period)

# X = []
# Y = []

# for i in range(n_lookback, len(y) - n_forecast + 1):
#     X.append(y[i - n_lookback: i])
#     Y.append(y[i: i + n_forecast])

# X = np.array(X)
# Y = np.array(Y)

# # fit the model
# model = Sequential()
# model.add(LSTM(units=50, return_sequences=True, input_shape=(n_lookback, 1)))
# model.add(LSTM(units=50))
# model.add(Dense(n_forecast))

# model.compile(loss='mean_squared_error', optimizer='adam')
# model.fit(X, Y, epochs=3, batch_size=32, verbose=0)

# # generate the forecasts
# X_ = y[- n_lookback:]  # last available input sequence
# X_ = X_.reshape(1, n_lookback, 1)

# Y_ = model.predict(X_).reshape(-1, 1)
# Y_ = scaler.inverse_transform(Y_)

# # organize the results in a data frame
# df_past = df[['Close']].reset_index()
# df_past.rename(columns={'index': 'Date', 'Close': 'Actual'}, inplace=True)
# df_past['Date'] = pd.to_datetime(df_past['Date'])
# df_past['Forecast'] = np.nan
# df_past['Forecast'].iloc[-1] = df_past['Actual'].iloc[-1]

# df_future = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
# df_future['Date'] = pd.date_range(start=df_past['Date'].iloc[-1] + pd.Timedelta(days=1), periods=n_forecast)
# df_future['Forecast'] = Y_.flatten()
# df_future['Actual'] = np.nan

# results = df_past.append(df_future).set_index('Date')

# # plot the results
# results.plot(title='Oil & Gas E&P', figsize=(16,8))

In [18]:
# Create a function to run the LSTM learning model for each industry
def learning_model(df):
    # download the data
    y = df['Adj Close'].fillna(method='ffill')
    y = y.values.reshape(-1, 1)

    # scale the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(y)
    y = scaler.transform(y)

    # generate the input and output sequences
    n_lookback = 365  # length of input sequences (lookback period)
    n_forecast = 180  # length of output sequences (forecast period)

    X = []
    Y = []

    for i in range(n_lookback, len(y) - n_forecast + 1):
        X.append(y[i - n_lookback: i])
        Y.append(y[i: i + n_forecast])

    X = np.array(X)
    Y = np.array(Y)

    # fit the model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(n_lookback, 1)))
    model.add(LSTM(units=50))
    model.add(Dense(n_forecast))

    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X, Y, epochs=3, batch_size=32, verbose=0)

    # generate the forecasts
    X_ = y[- n_lookback:]  # last available input sequence
    X_ = X_.reshape(1, n_lookback, 1)

    Y_ = model.predict(X_).reshape(-1, 1)
    Y_ = scaler.inverse_transform(Y_)

    # organize the results in a data frame
    df_past = df[['Close']].reset_index()
    df_past.rename(columns={'index': 'Date', 'Close': 'Actual'}, inplace=True)
    df_past['Date'] = pd.to_datetime(df_past['Date'])
    df_past['Forecast'] = np.nan
    df_past['Forecast'].iloc[-1] = df_past['Actual'].iloc[-1]

    df_future = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
    df_future['Date'] = pd.date_range(start=df_past['Date'].iloc[-1] + pd.Timedelta(days=1), periods=n_forecast)
    df_future['Forecast'] = Y_.flatten()
    df_future['Actual'] = np.nan

    results = df_past.append(df_future).set_index('Date')
    
    # Calculate rate of return on investment based on projected future stock price
    return_rate = (results['Forecast'][-1] - df['Adj Close'][-1]) / df['Adj Close'][-1]
    
    return return_rate

    # plot the results - optional
    #results.plot(title='VIST', figsize=(16,8))

In [19]:
# Run learning_model on industry dataframe
learning_model(df)

1/1 [==============================] - 1s 929ms/step


-0.39141200434834056

In [20]:
# # Create new dataframe to house all company stock symbols and their associated growth rates
# industry_roi_df = pd.DataFrame(industries, columns=['Industry'])

# # Iterate through each unique stock symbol and filter the historical stock data for each company
# for index, industry in industry_roi_df.iterrows():
#     industry = industry.values[0]
#     industry_df = industry_data[industry_data.industry == industry]
#     print(f'The index currently running is {index}, which is the {industry} industry.')
    
#     # Run the learning model function for each company
#     industry_roi_df.loc[index, 'ROI'] = learning_model(industry_df)

In [21]:
# industry_roi_df